In [2]:
# ===============================================
# GET TEST DATA FOR FALL DETECTION MODEL (WITH MAG FEATURES)
# ===============================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def get_fall_detection_test_data(filepath='/content/DataSet.csv', time_steps=50, test_size=0.2, random_state=42):
    # Load CSV
    df = pd.read_csv(filepath, header=[0,1])
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    current_cols = df.columns.tolist()

    # Rename accelerometer columns if موجودة
    wrist_accel_x_col = 'WristAccelerometer_x-axis (g)'
    if wrist_accel_x_col in current_cols:
        x_index = current_cols.index(wrist_accel_x_col)
        df.rename(columns={
            current_cols[x_index + 1]: 'WristAccelerometer_y-axis (g)',
            current_cols[x_index + 2]: 'WristAccelerometer_z-axis (g)',
        }, inplace=True)

    # Rename gyroscope columns if موجودة
    wrist_gyro_x_col = 'WristAngularVelocity_x-axis (deg/s)'
    if wrist_gyro_x_col in current_cols:
        x_index = current_cols.index(wrist_gyro_x_col)
        df.rename(columns={
            current_cols[x_index + 1]: 'WristAngularVelocity_y-axis (deg/s)',
            current_cols[x_index + 2]: 'WristAngularVelocity_z-axis (deg/s)',
        }, inplace=True)

    # Activity column
    activity_col = [col for col in df.columns if col.startswith('Activity')][0]

    # Original 6 features
    FEATURE_COLS = [
        'WristAccelerometer_x-axis (g)',
        'WristAccelerometer_y-axis (g)',
        'WristAccelerometer_z-axis (g)',
        'WristAngularVelocity_x-axis (deg/s)',
        'WristAngularVelocity_y-axis (deg/s)',
        'WristAngularVelocity_z-axis (deg/s)'
    ]

    # ======================================
    # ➕ NEW: Add Magnitude Columns
    # ======================================
    df['Acc_mag'] = np.sqrt(
        df['WristAccelerometer_x-axis (g)']**2 +
        df['WristAccelerometer_y-axis (g)']**2 +
        df['WristAccelerometer_z-axis (g)']**2
    )

    df['Gyro_mag'] = np.sqrt(
        df['WristAngularVelocity_x-axis (deg/s)']**2 +
        df['WristAngularVelocity_y-axis (deg/s)']**2 +
        df['WristAngularVelocity_z-axis (deg/s)']**2
    )

    # Add them to feature list
    FEATURE_COLS += ['Acc_mag', 'Gyro_mag']

    # ======================================
    # Target
    # ======================================
    FALL_ACTIVITY_CODES = [7, 8, 9, 10, 11]
    df['Target'] = df[activity_col].apply(lambda x: 1 if x in FALL_ACTIVITY_CODES else 0)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[FEATURE_COLS].values)
    y_target = df['Target'].values

    # Create sequences
    def create_sequences(data, target, time_steps):
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), :])
            y.append(target[i + time_steps - 1])
        return np.array(X), np.array(y)

    X_sequences, y_labels = create_sequences(X_scaled, y_target, time_steps)

    # Split into train/test
    _, X_test, _, y_test = train_test_split(
        X_sequences, y_labels,
        test_size=test_size, stratify=y_labels, random_state=random_state
    )

    print(f"✓ Test data ready. X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
    print(f"Features used: {len(FEATURE_COLS)} → {FEATURE_COLS}")

    return X_test, y_test


# ======= USAGE =======
filename = "/content/DataSet.csv"
X_test, y_test = get_fall_detection_test_data(filename)

# Save test data
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)
print("✓ Test data saved as X_test.npy and y_test.npy")


✓ Test data ready. X_test shape: (36517, 50, 8), y_test shape: (36517,)
Features used: 8 → ['WristAccelerometer_x-axis (g)', 'WristAccelerometer_y-axis (g)', 'WristAccelerometer_z-axis (g)', 'WristAngularVelocity_x-axis (deg/s)', 'WristAngularVelocity_y-axis (deg/s)', 'WristAngularVelocity_z-axis (deg/s)', 'Acc_mag', 'Gyro_mag']
✓ Test data saved as X_test.npy and y_test.npy


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("bilstm_advanced_final.keras")
print(model.input_shape)


(None, 60, 8)


In [4]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score

# -----------------------------
# YOUR MODEL PATHS
# -----------------------------
model_paths = [
    "/content/best_bilstm_advanced.keras",
    "/content/best_bilstm_fall_detection_model.keras",
    "/content/best_fall_detection_model.keras",
    "/content/bilstm_advanced_final.keras",
    "/content/bilstm_fall_detection_model.keras",
    "/content/fall_detection_fixed.h5"
]

# Load valid models
selected_models = []
for path in model_paths:
    try:
        model = load_model(path)
        selected_models.append(model)
        print(f"Loaded: {path}")
    except Exception as e:
        print(f"Error loading {path}: {e}")

# Predict probabilities for each model
preds_list = []
for m in selected_models:
    p = m.predict(X_test)
    preds_list.append(p.flatten())

preds_array = np.column_stack(preds_list)

# Weighted averaging
weights = np.ones(len(selected_models)) / len(selected_models)
ensemble_prob = np.dot(preds_array, weights)

print("\nSample ensemble probabilities:", ensemble_prob[:10])

# ----------------------
# Convert to class (0/1)
# ----------------------
threshold = 0.5
ensemble_pred = (ensemble_prob >= threshold).astype(int)

# ----------------------
# Metrics
# ----------------------
precision = precision_score(y_test, ensemble_pred, zero_division=0)
recall = recall_score(y_test, ensemble_pred, zero_division=0)
f1 = f1_score(y_test, ensemble_pred, zero_division=0)

print("\n========== ENSEMBLE METRICS ==========")
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1-score:", round(f1, 4))
print("======================================")


Loaded: /content/best_bilstm_advanced.keras
Loaded: /content/best_bilstm_fall_detection_model.keras
Loaded: /content/best_fall_detection_model.keras
Loaded: /content/bilstm_advanced_final.keras


Loaded: /content/bilstm_fall_detection_model.keras
Loaded: /content/fall_detection_fixed.h5
1142/1142 ━━━━━━━━━━━━━━━━━━━━ 99s 86ms/step
1142/1142 ━━━━━━━━━━━━━━━━━━━━ 62s 54ms/step
1142/1142 ━━━━━━━━━━━━━━━━━━━━ 47s 40ms/step
1142/1142 ━━━━━━━━━━━━━━━━━━━━ 98s 85ms/step
1142/1142 ━━━━━━━━━━━━━━━━━━━━ 62s 53ms/step
1142/1142 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step

Sample ensemble probabilities: [6.37027732e-04 9.87044324e-01 2.40419872e-01 8.47832253e-01
 2.69770537e-02 9.16218797e-01 1.16805204e-01 3.36966681e-01
 4.44323312e-01 8.68667352e-02]

========== ENSEMBLE METRICS ==========
Precision: 0.8959
Recall: 0.6512
F1-score: 0.7542


In [13]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score

# -----------------------------
# YOUR MODEL PATHS
# -----------------------------
model_paths = [
    "/content/best_bilstm_advanced.keras",
    "/content/best_bilstm_fall_detection_model.keras",

    "/content/fall_detection_fixed.h5"
]

# Load valid models
selected_models = []
for path in model_paths:
    try:
        model = load_model(path)
        selected_models.append(model)
        print(f"Loaded: {path}")
    except Exception as e:
        print(f"Error loading {path}: {e}")

# Predict probabilities for each model
preds_list = []
for m in selected_models:
    p = m.predict(X_test)
    preds_list.append(p.flatten())

preds_array = np.column_stack(preds_list)

# Weighted averaging
weights = np.ones(len(selected_models)) / len(selected_models)
ensemble_prob = np.dot(preds_array, weights)

print("\nSample ensemble probabilities:", ensemble_prob[:10])

# ----------------------
# Convert to class (0/1)
# ----------------------
threshold = 0.5
ensemble_pred = (ensemble_prob >= threshold).astype(int)

# ----------------------
# Metrics
# ----------------------
precision = precision_score(y_test, ensemble_pred, zero_division=0)
recall = recall_score(y_test, ensemble_pred, zero_division=0)
f1 = f1_score(y_test, ensemble_pred, zero_division=0)

print("\n========== ENSEMBLE METRICS ==========")
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1-score:", round(f1, 4))
print("======================================")


Loaded: /content/best_bilstm_advanced.keras


Loaded: /content/best_bilstm_fall_detection_model.keras
Loaded: /content/fall_detection_fixed.h5
9208/9208 ━━━━━━━━━━━━━━━━━━━━ 812s 88ms/step
9208/9208 ━━━━━━━━━━━━━━━━━━━━ 512s 55ms/step
9208/9208 ━━━━━━━━━━━━━━━━━━━━ 113s 12ms/step

Sample ensemble probabilities: [0.38799368 0.07376124 0.06551781 0.0107462  0.0266843  0.05867904
 0.08185124 0.15420164 0.2151887  0.22232504]

========== ENSEMBLE METRICS ==========
Precision: 0.9785
Recall: 0.9386
F1-score: 0.9582
